In [1]:
import torch
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder

from google.colab import drive
drive.mount('/content/drive')

cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
class VGG(torch.nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        #self.classifier = torch.nn.Linear(512, 10)
        self.classifier = nn.Sequential(
            nn.Linear(25088,4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096,4096),
            nn.ReLU(),
            torch.nn.Dropout(p=0.5),
            nn.Linear(4096,5)
        )

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [torch.nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [torch.nn.Conv2d(in_channels, x, kernel_size=3, padding=1), torch.nn.BatchNorm2d(x),
                           torch.nn.ReLU(inplace=True)]
                in_channels = x
        layers += [torch.nn.AvgPool2d(kernel_size=1, stride=1)]
        return torch.nn.Sequential(*layers)

In [0]:
vgg = VGG("VGG11").cuda()

transform = transforms.Compose([
        transforms.Resize((224,224)),
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ]
)


#flower_dataset = ImageFolder(root="./flowers", transform = transform)
flower_dataset = ImageFolder(root="/content/drive/My Drive/colab/flowers", transform = transform)

train_size = int(0.8 * len(flower_dataset))
test_size = len(flower_dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(flower_dataset, [train_size, test_size])

flower_train = DataLoader(train_dataset,batch_size = 128 , shuffle = True , num_workers= 0)
flower_test = DataLoader(test_dataset,batch_size = 128 , shuffle = True , num_workers= 0)

classes = ('daisy','dandelion','rose','sunflower','tulip')

In [4]:
# Define a Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg.parameters(),lr=0.001 , momentum= 0.9)

# Train the network
for epoch in range(15):
    for i,data in enumerate(flower_train,0):

        inputs,labels =data

        inputs = inputs.cuda()
        labels = labels.cuda()

        optimizer.zero_grad()
        outputs = vgg(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        print('代數:%d  次數:%5d loss: %.3f' %(epoch + 1, i + 1, loss.item()))

print('Finished Training')

代數:1  次數:    1 loss: 1.607
代數:1  次數:    2 loss: 1.628
代數:1  次數:    3 loss: 1.624
代數:1  次數:    4 loss: 1.628
代數:1  次數:    5 loss: 1.582
代數:1  次數:    6 loss: 1.616
代數:1  次數:    7 loss: 1.498
代數:1  次數:    8 loss: 1.584
代數:1  次數:    9 loss: 1.562
代數:1  次數:   10 loss: 1.568
代數:1  次數:   11 loss: 1.547
代數:1  次數:   12 loss: 1.510
代數:1  次數:   13 loss: 1.550
代數:1  次數:   14 loss: 1.525
代數:1  次數:   15 loss: 1.457
代數:1  次數:   16 loss: 1.498
代數:1  次數:   17 loss: 1.496
代數:1  次數:   18 loss: 1.485
代數:1  次數:   19 loss: 1.485
代數:1  次數:   20 loss: 1.405
代數:1  次數:   21 loss: 1.365
代數:1  次數:   22 loss: 1.340
代數:1  次數:   23 loss: 1.408
代數:1  次數:   24 loss: 1.406
代數:1  次數:   25 loss: 1.352
代數:1  次數:   26 loss: 1.295
代數:1  次數:   27 loss: 1.336
代數:1  次數:   28 loss: 1.582
代數:2  次數:    1 loss: 1.259
代數:2  次數:    2 loss: 1.446
代數:2  次數:    3 loss: 1.301
代數:2  次數:    4 loss: 1.335
代數:2  次數:    5 loss: 1.353
代數:2  次數:    6 loss: 1.252
代數:2  次數:    7 loss: 1.285
代數:2  次數:    8 loss: 1.148
代數:2  次數:    9 loss: 1.201
代

In [5]:
correct = 0
total = 0
with torch.no_grad():
    for data in flower_test:
        images, labels = data

        images = images.cuda()
        labels = labels.cuda()

        outputs = vgg(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('測試資料成功率: %d %%' % (
    100 * correct / total))

測試資料成功率: 71 %


In [6]:
class_correct = list(0. for i in range(5))
class_total = list(0. for i in range(5))
with torch.no_grad():
    for data in flower_test:
        images, labels = data

        images = images.cuda()
        labels = labels.cuda()

        outputs = vgg(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(5):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of daisy : 75 %
Accuracy of dandelion : 100 %
Accuracy of  rose : 87 %
Accuracy of sunflower : 25 %
Accuracy of tulip : 66 %
